In [10]:
from keras.models import Sequential
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

import sys
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
"""
 0: airplane
 1: automobile
 2: bird
 3: cat
 4: deer
 5: dog
 6: frog
 7: horse
 8: ship
 9: truck
 """

'\n 0: airplane\n 1: automobile\n 2: bird\n 3: cat\n 4: deer\n 5: dog\n 6: frog\n 7: horse\n 8: ship\n 9: truck\n '

In [3]:
(x_train, y_train) , (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [5]:
#data preprocessing(whiten)
mean = [0,0,0]
std = [0,0,0]
newX_train = np.ones(x_train.shape)
newX_test = np.ones(x_test.shape)

#train set에 있는 데이터로 mean과 std를 구함
for i in range(3): #channel = 3
  mean[i] = np.mean(x_train[ : , : , : , i])
  std[i] = np.std(x_train[ : , : , : ,i])

#정규화하기 N(0,1)
for i in range(3):
  newX_train[ : , : , : , i] = x_train[ : , : , : ,i] - mean[i]
  newX_train[ : , : , : , i] = newX_train[ : , : , : ,i] / std[i]
  newX_test[ : , : , : , i] = x_test[ : , : , : ,i] - mean[i]
  newX_test[ : , : , : , i] = newX_test[ : , : , : ,i] / std[i]

X_train = newX_train
X_test = newX_test
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [8]:
def define_model():
  #1 layer
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
 
  #2 layer
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))
  
  #3 layer
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.4))
 
  #4 layer
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	model.add(Dense(10, activation='softmax'))
 
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [9]:
#plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [11]:
# run the test harness for evaluating a model

# PM.04:51                                                                                          

# load dataset -> upper

# define model
model = define_model()
# create data generator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(X_train, Y_train, batch_size=64)
# fit model
steps = int(X_train.shape[0] / 64)
history = model.fit_generator(it_train, steps_per_epoch=steps, epochs=400, validation_data=(X_test, Y_test), verbose=0)
# evaluate model
_, acc = model.evaluate(X_test, Y_test, verbose=0)
print('> %.3f' % (acc * 100.0))
# learning curves
summarize_diagnostics(history)

Instructions for updating:
Please use Model.fit, which supports generators.


KeyboardInterrupt: ignored